# RSA 原理與實作

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import binascii
import random

### 轉碼

* UTF-8 → Decimal 

In [2]:
def string_to_int(string):
    byte = string.encode()               # string to byte
    hexadecimal = binascii.hexlify(byte) # byte to hex
    decimal = int(hexadecimal, 16)       # hex to dec
    return decimal

* Decimal  → UTF-8

In [3]:
def int_to_string(int):
    hexadecimal = hex(int)               # dec to hex
    byte = hexadecimal[2:].encode()      # hex to byte (type)
    byte = binascii.unhexlify(byte)      # hex to byte (coding)
    string = byte.decode('utf-8')        # byte to utf-8
    return string

In [4]:
string = 'Hello World! 你好！'
integer = string_to_int(string)
print(integer)
message = int_to_string(integer)
print(message)

27086628833817823194713031788698839469377765346622593
Hello World! 你好！


### 質因數分解

In [5]:
def factorize(x):
    is_prime = True
    for i in sp.primerange(0, x ** 0.5 + 1):
        if x % i == 0:
            j = x // i
            is_prime = False
            return [i, j]
    if is_prime:
        return [0, x]

In [6]:
print(factorize(2 ** 10))

[2, 512]


In [7]:
def factorize2(x):
    fact_list = list()
    if sp.isprime(x) == False:
        fact = 1
        while fact != 0:
            fact, x = factorize(x)    
            if fact != 0:
                fact_list.append(fact)
        if x != 1:
            fact_list.append(x)
    else:
        fact_list.append(x)
    return fact_list

In [8]:
print(factorize2(2 ** 10))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### 最大公因數

In [9]:
def greatest_common_divisor(a, b):
    if b == 0:
        return a
    else:
        return greatest_common_divisor(b, a % b)

### 歐拉函數

In [10]:
def euler_function(p, q):
    return (p - 1) * (q - 1)

### 加密函數

In [11]:
def encode(plaintext, public_key, n):
    cyphertext = (plaintext ** public_key) % n    
    return cyphertext

### 解密函數

In [12]:
def decode(cyphertext, private_key, n):
    plaintext = (cyphertext ** private_key) % n
    return plaintext

In [13]:
def decode2(cyphertext, private_key_fact_list, n):
    for i in private_key_fact_list:
        cyphertext = (cyphertext ** i) % n
    plaintext = cyphertext
    return plaintext

### 產生鑰匙

In [14]:
def generate_key():
    public_key, private_key = 0, 0
    prime = list(sp.primerange((10 ** 3), (10 ** 4)))
    while (public_key == 0) or (private_key == 0):
        p, q = random.choices(prime, k=2)
        n = p * q
        o = euler_function(p, q)
        public_key, private_key = factorize(o * 10 + 1)
        gcd = greatest_common_divisor(public_key, o)
        remainder = public_key * private_key % o
        print('gcd: %i' % gcd)
        print('remainder: %i' % remainder)
        if gcd == 1 and remainder == 1:
            print('generate key succes\n')
            break
        else:
            print('generate key fail\n')
    return p, q, public_key, private_key, n

In [18]:
p, q, public_key, private_key, n = generate_key()
private_key_fact_list = factorize2(private_key) # 分解 private key，避免解密時間過長
print('p: %i' % p)
print('q: %i' % q)
print('public key: %i' % public_key)
print('private key: %i = %s' % (private_key, str(private_key_fact_list)))
print('n: %i' % n)

gcd: 1
remainder: 1
generate key succes

p: 6353
q: 6073
public key: 7
private key: 55099063 = [211, 223, 1171]
n: 38581769


### 加密

In [19]:
plaintext = input()
plaintext_list = list()
cyphertext_list = list()
for i in plaintext:
    integer = string_to_int(i)
    plaintext_list.append(integer)
    cyphertext = encode(integer, public_key, n)
    cyphertext_list.append(cyphertext)
print('plain text: %s' % str(plaintext_list))
print('cypher text: %s' % str(cyphertext_list))

Hello World! 你好！
plain text: [72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33, 32, 14990752, 15050173, 15711361]
cypher text: [8954361, 7698516, 9819119, 9819119, 31437685, 21963958, 32850738, 31437685, 32139848, 9819119, 28674207, 24170001, 21963958, 10218399, 16696036, 6748821]


### 解密

In [20]:
message = ''
for i in cyphertext_list:
    plaintext = decode2(i, private_key_fact_list, n)
    message += int_to_string(plaintext)
print(message)

Hello World! 你好！
